This is the code to combine the libraries (file: 'combined-XChem-libraries.csv').
Not to map them.

In [ ]:
import pandas as pd
from rdkit.Chem.Draw import MolsToGridImage
from rdkit import Chem

dfs = []

In [ ]:
keepers = {'Library Name': 'Id', 'SMILES': 'SMILES'}
lib_name = 'York3D'
df = pd.read_excel('York3d_DMSO_d6 platemap_corrected.xlsx')

df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'Catalog ID': 'Id', 'Smiles': 'SMILES', 'Chemical name': 'Name'}
lib_name = 'DSiPoised'
df = pd.read_excel('../Fragmenstein_collection/rescoring/DSiP_Batch-4_Plate1308634_PO8005504_DMSO.xlsx')

df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'COMPOUND_ID': 'Id', 'Smiles': 'SMILES', 'IUPAC_NAME': 'Name'}
lib_name = 'Euopen'
filename = '/Users/matteo/Downloads/Euopen_screen_with_smiles.xlsx'

df = pd.read_excel(filename)
df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'LDS ID': 'Id', 'Smiles': 'SMILES'}
lib_name = 'Leed3D'
filename = '/Users/matteo/Downloads/Leeds3D.xlsx'

df = pd.read_excel(filename)
df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'Catalog_ID': 'Id', 'Smile': 'SMILES', 'Name': 'Name'}
lib_name = 'Leed3D'
filename = '/Users/matteo/Downloads/H92015274.xlsx'

df = pd.read_excel(filename)
df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'Compound Name (Internal)': 'Id', 'SMILES': 'SMILES'}
lib_name = 'Cambridge3D'
filename = '/Users/matteo/Downloads/SKiddLibrary.xlsx'

df = pd.read_excel(filename)
df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'Code': 'Id', 'SMILES': 'SMILES'}
lib_name = 'FragLite'
filename = '/Users/matteo/Downloads/ECHO_FragLiteSET1.xlsx'

df = pd.read_excel(filename)
df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'Library Name': 'Id', 'SMILES': 'SMILES'}
lib_name = 'PepLite'
filename = '/Users/matteo/Downloads/ECHO_PepLite_DMSO.xlsx'

df = pd.read_excel(filename)
df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'catalogID': 'Id', 'Smile': 'SMILES', 'Name': 'Name'}
lib_name = 'SpotXplorer'
filename = '/Users/matteo/Downloads/Diamond_1536-well_6_plates_3copies.xlsx'

df = pd.read_excel(filename)
df = df[df.ID.str.contains('SF')]
df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
keepers = {'catalogID': 'Id', 'Smile': 'SMILES', 'Name': 'Name'}
lib_name = 'CovHetFrags'
filename = '/Users/matteo/Downloads/Diamond_1536-well_6_plates_3copies.xlsx'

df = pd.read_excel(filename)
df = df[df.ID.str.contains('Cov_HetLib')]
df = df.rename(columns=keepers)[keepers.values()]
df['library'] = lib_name
dfs.append(df)

In [ ]:
combo = pd.concat(dfs)

In [ ]:
# acetate causes issues...
combo['raw_SMILES'] = combo['SMILES']
combo['SMILES'] = combo.SMILES.apply(lambda v: sorted(v.split('.'), key=len, reverse=True)[0])

In [ ]:
from rdkit import Chem
combo['inchikey'] = combo.SMILES.apply(Chem.MolFromSmiles).apply(Chem.MolToInchiKey)

In [ ]:
from collections import defaultdict
# technically could be done w/ the reverse of a pivot table or meld... but shmeh.

alt = defaultdict(list)
for i, row in combo.loc[combo.duplicated('inchikey', False)].iterrows():
    alt[row.inchikey].append(row.library)
    
combo['also_in'] = ''

for i, row in combo.loc[combo.duplicated('inchikey', False)].iterrows():
    combo.at[i, 'also_in'] = ','.join(sorted(set(alt[row.inchikey]) - {row.library}))

In [ ]:
# No screamcaps

def fix(name):
    if name == name.upper():
        return name.lower()
    return name

combo['Murcko_name'] = combo.Murcko_name.fillna('').apply(fix)

In [ ]:
combo[['Id', 'library', 'also_in', 'Name', 'SMILES', 'raw_SMILES', 'inchikey', 'N_heavy_atoms',
       'Murcko', 'Murcko_name']].to_csv('combined-XChem-libraries.csv')